In [44]:
#Imports
import pandas as pd
import numpy as np
import requests
import re

In [45]:
#Parameters
url = 'https://raw.githubusercontent.com/LuizFelipeAG/awaritasks/main/calculadora_aluguel/manipulacao/dadosrj.csv'
MAPQUEST_KEY='BQDX4RkZleZLqQ9i7KAolyH5Hz4AGtrb'
MAPQUEST_URL='http://open.mapquestapi.com/geocoding/v1/address?key={}&location={}'

In [46]:
#Functions
def endereco2latlon(endereco):
    mapquest_request_url=MAPQUEST_URL.format(MAPQUEST_KEY, endereco)
    response = requests.get(mapquest_request_url).json()
    endereco_lat = response['results'][0]['locations'][0]['latLng']['lat']
    endereco_lng = response['results'][0]['locations'][0]['latLng']['lng']
    return (endereco_lat, endereco_lng)

def extrair_latitude(coordenada):
  if coordenada != '':
    lat = float(coordenada.split(',')[0].replace('(', ''))
  else:
    lat = np.nan
  return lat

def extrair_longitude(coordenada):
  if coordenada != '':
    lon = float(coordenada.split(',')[1].replace(')', ''))
  else:
    lon = np.nan
  return lon

def extrair_area(entrada):
  """Criar função que possua como entrada um texto e retorne a área contida no
  mesmo. Retornar np.nan se for ausentes. Alguns exemplos:
      >>> extrair_area('79 m² construídos1 quarto')
      >>> 79
      >>> extrair_area('280 m² construídos')
      >>> 280
      >>> extrair_area('2,600 m² construídos'
      >>> 2600
      >>> extrair_area('2 quartos')
      >>> np.nan
      """
  if ' m²' in entrada:
    area = entrada.split(' m²')[0]
    area = area.replace(',', '')
  else:
    area = np.nan
  return area

def extrair_quartos(entrada):
  """Criar função que possua como entrada um texto e retorne uma tupla com a 
  área e o quarto. Retornar np.nan se um dos dois for ausentes. Alguns exemplos:
      >>> extrair_quarto('79 m² construídos1 quarto')
      >>> 1
      >>> extrair_quarto('280 m² construídos')
      >>> np.nan
      >>> extrair_quarto('2,600 m² construídos'
      >>> np.nan
      >>> extrair_quarto('2 quartos')
      >>> 2"""
  if 'quarto' in entrada:
    n_quartos = re.findall('(\d*) quarto', entrada)[0]
  else:
    n_quartos = np.nan
  return n_quartos

In [47]:
df = pd.read_csv(url, thousands='.', encoding='latin-1')

In [48]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel
0,centro,50 m² construídos1 quarto,"Rua Senador Dantas, Centro, Rio De Janeiro Cen...",970
1,centro,28 m² construídos1 quarto,"Rua Das Marrecas, Centro, Rio De Janeiro Centr...",1224
2,centro,38 m² construídos1 quarto,"Rua De Santana, Centro, Rio De Janeiro Centro,...",1000
3,centro,62 m² construídos2 quartos,"Rua Riachuelo, Centro, Rio De Janeiro Centro, ...",2200
4,centro,48 m² construídos2 quartos,"Rua João Neves Da Fontoura, Centro, Rio De Jan...",1001


(39.78373, -100.445882)

In [50]:
df['coordenadas']=df['enderecos_lista'].apply(endereco2latlon).astype(str)

In [51]:
#aplicando funcoes para criar as colunas latitude e longitude
df['latitude']=df['coordenadas'].apply(extrair_latitude).astype(float)
df['longitude']=df['coordenadas'].apply(extrair_longitude).astype(float)

In [52]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel,coordenadas,latitude,longitude
0,centro,50 m² construídos1 quarto,"Rua Senador Dantas, Centro, Rio De Janeiro Cen...",970,"(-22.910191, -43.17739)",-22.910191,-43.177390
1,centro,28 m² construídos1 quarto,"Rua Das Marrecas, Centro, Rio De Janeiro Centr...",1224,"(-22.911846, -43.178483)",-22.911846,-43.178483
2,centro,38 m² construídos1 quarto,"Rua De Santana, Centro, Rio De Janeiro Centro,...",1000,"(-22.371619, -41.784483)",-22.371619,-41.784483
3,centro,62 m² construídos2 quartos,"Rua Riachuelo, Centro, Rio De Janeiro Centro, ...",2200,"(-22.912523, -43.191971)",-22.912523,-43.191971
4,centro,48 m² construídos2 quartos,"Rua João Neves Da Fontoura, Centro, Rio De Jan...",1001,"(39.78373, -100.445882)",39.783730,-100.445882


In [53]:
# removendo a coluna coordenadas
df = df.drop('coordenadas', axis=1)

In [54]:
# Aplicando funcao para criar a coluna area
df['area']=df['area_quartos'].apply(extrair_area).astype(float)

In [55]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel,latitude,longitude,area
0,centro,50 m² construídos1 quarto,"Rua Senador Dantas, Centro, Rio De Janeiro Cen...",970,-22.910191,-43.177390,50.0
1,centro,28 m² construídos1 quarto,"Rua Das Marrecas, Centro, Rio De Janeiro Centr...",1224,-22.911846,-43.178483,28.0
2,centro,38 m² construídos1 quarto,"Rua De Santana, Centro, Rio De Janeiro Centro,...",1000,-22.371619,-41.784483,38.0
3,centro,62 m² construídos2 quartos,"Rua Riachuelo, Centro, Rio De Janeiro Centro, ...",2200,-22.912523,-43.191971,62.0
4,centro,48 m² construídos2 quartos,"Rua João Neves Da Fontoura, Centro, Rio De Jan...",1001,39.783730,-100.445882,48.0


In [56]:
# Aplicando funcao para criar a coluna quartos
df['quartos'] = df['area_quartos'].apply(extrair_quartos).astype(float)

In [57]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel,latitude,longitude,area,quartos
0,centro,50 m² construídos1 quarto,"Rua Senador Dantas, Centro, Rio De Janeiro Cen...",970,-22.910191,-43.177390,50.0,1.0
1,centro,28 m² construídos1 quarto,"Rua Das Marrecas, Centro, Rio De Janeiro Centr...",1224,-22.911846,-43.178483,28.0,1.0
2,centro,38 m² construídos1 quarto,"Rua De Santana, Centro, Rio De Janeiro Centro,...",1000,-22.371619,-41.784483,38.0,1.0
3,centro,62 m² construídos2 quartos,"Rua Riachuelo, Centro, Rio De Janeiro Centro, ...",2200,-22.912523,-43.191971,62.0,2.0
4,centro,48 m² construídos2 quartos,"Rua João Neves Da Fontoura, Centro, Rio De Jan...",1001,39.783730,-100.445882,48.0,2.0


In [58]:
# removendo a coluna area_quartos
df = df.drop('area_quartos', axis=1)

In [59]:
# reposicionando as colunas
cols = ['zona', 'enderecos_lista','latitude', 'longitude', 'area', 'quartos', 'aluguel']

In [60]:
df=df[cols]

In [61]:
df.head()

,zona,enderecos_lista,latitude,longitude,area,quartos,aluguel
0,centro,"Rua Senador Dantas, Centro, Rio De Janeiro Cen...",-22.910191,-43.177390,50.0,1.0,970
1,centro,"Rua Das Marrecas, Centro, Rio De Janeiro Centr...",-22.911846,-43.178483,28.0,1.0,1224
2,centro,"Rua De Santana, Centro, Rio De Janeiro Centro,...",-22.371619,-41.784483,38.0,1.0,1000
3,centro,"Rua Riachuelo, Centro, Rio De Janeiro Centro, ...",-22.912523,-43.191971,62.0,2.0,2200
4,centro,"Rua João Neves Da Fontoura, Centro, Rio De Jan...",39.783730,-100.445882,48.0,2.0,1001


In [62]:
# antes de movermos para preprocessamentos voltados à valores ausentes, vamos 
# salvar os dados do jeito como estão
df.to_csv('dados_aluguel_ml_rj_preprocessado.csv', index=False)

In [64]:
# Index=false usado anteriormente para que o csv seja carregado sem necessitar argumentos adicionais
df = pd.read_csv('dados_aluguel_ml_rj_preprocessado.csv')

In [65]:
df

,zona,enderecos_lista,latitude,longitude,area,quartos,aluguel
0,centro,"Rua Senador Dantas, Centro, Rio De Janeiro Cen...",-22.910191,-43.177390,50.0,1.0,970
1,centro,"Rua Das Marrecas, Centro, Rio De Janeiro Centr...",-22.911846,-43.178483,28.0,1.0,1224
2,centro,"Rua De Santana, Centro, Rio De Janeiro Centro,...",-22.371619,-41.784483,38.0,1.0,1000
3,centro,"Rua Riachuelo, Centro, Rio De Janeiro Centro, ...",-22.912523,-43.191971,62.0,2.0,2200
4,centro,"Rua João Neves Da Fontoura, Centro, Rio De Jan...",39.783730,-100.445882,48.0,2.0,1001
...,...,...,...,...,...,...,...
1339,zona-sul,"Rua Nina Rodrigues, Jardim Botânico, Rio De Ja...",39.783730,-100.445882,150.0,4.0,6800
1340,zona-sul,"Rua Canning, Ipanema, Rio De Janeiro Zona Sul,...",-22.984896,-43.195214,84.0,2.0,5200
1341,zona-sul,"Rua Joaquim Murtinho, Santa Teresa, Rio De Jan...",39.783730,-100.445882,110.0,3.0,2500
1342,zona-sul,"Avenida Visconde De Albuquerque, Leblon, Rio D...",39.783730,-100.445882,200.0,4.0,13800


### Lidando com dados ausentes

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1344 entries, 0 to 1343
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   zona             1344 non-null   object 
 1   enderecos_lista  1344 non-null   object 
 2   latitude         1344 non-null   float64
 3   longitude        1344 non-null   float64
 4   area             1344 non-null   float64
 5   quartos          1101 non-null   float64
 6   aluguel          1344 non-null   int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 73.6+ KB


In [67]:
df.describe()

,latitude,longitude,area,quartos,aluguel
count,1344.000000,1344.000000,1344.000000,1101.000000,1344.000000
mean,1.574650,-65.563985,138.544643,2.057221,5125.079613
std,30.603159,27.938364,173.341891,0.900200,10142.633778
min,-22.998618,-100.445882,18.000000,1.000000,500.000000
25%,-22.920170,-100.445882,53.750000,1.000000,1343.500000
50%,-22.901056,-43.255659,77.000000,2.000000,2200.000000
75%,39.783730,-43.184127,134.000000,3.000000,3915.000000
max,39.783730,-41.784483,1299.000000,6.000000,103929.000000


In [68]:
df.median()

latitude      -22.901056
longitude     -43.255659
area           77.000000
quartos         2.000000
aluguel      2200.000000
dtype: float64

In [71]:
df_ready = df.fillna(df.median())

In [73]:
df_ready.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1344 entries, 0 to 1343
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   zona             1344 non-null   object 
 1   enderecos_lista  1344 non-null   object 
 2   latitude         1344 non-null   float64
 3   longitude        1344 non-null   float64
 4   area             1344 non-null   float64
 5   quartos          1344 non-null   float64
 6   aluguel          1344 non-null   int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 73.6+ KB


### O problema do Geolocation
A API do Mapquest não consegue localizar todos as coordenadas nos endereços listados. Ele insere uma coordenada padrão com latitude 39.783730 e longitude -100.445882 quando isso acontece.

In [74]:
df.loc[df['latitude'] == 39.783730]

,zona,enderecos_lista,latitude,longitude,area,quartos,aluguel
4,centro,"Rua João Neves Da Fontoura, Centro, Rio De Jan...",39.78373,-100.445882,48.0,2.0,1001
10,centro,"Rua Eleone De Almeida, Catumbi, Rio De Janeiro...",39.78373,-100.445882,50.0,2.0,1200
12,centro,"Rua Do Chichorro, Catumbi, Rio De Janeiro Cent...",39.78373,-100.445882,80.0,2.0,2163
14,centro,"Rua Do Chichorro, Catumbi, Rio De Janeiro Cent...",39.78373,-100.445882,78.0,2.0,1500
17,centro,"Rua Ubaldino Do Amaral, Centro, Rio De Janeiro...",39.78373,-100.445882,75.0,2.0,3000
...,...,...,...,...,...,...,...
1321,zona-sul,"Rua Paschoal Carlos Magno, Santa Teresa, Rio D...",39.78373,-100.445882,33.0,1.0,2200
1329,zona-sul,"Rua Presidente Alfonso López, Lagoa, Rio De Ja...",39.78373,-100.445882,95.0,3.0,3600
1339,zona-sul,"Rua Nina Rodrigues, Jardim Botânico, Rio De Ja...",39.78373,-100.445882,150.0,4.0,6800
1341,zona-sul,"Rua Joaquim Murtinho, Santa Teresa, Rio De Jan...",39.78373,-100.445882,110.0,3.0,2500


In [84]:
endereco2latlon('Rua João Neves Da Fontoura, Centro, Rio De Janeiro, Centro, Rio De Janeiro')

(39.78373, -100.445882)

In [86]:
#Adicionar o CEP também não ajuda
endereco2latlon('Rua João Neves Da Fontoura, Centro, Rio De Janeiro, Centro, Rio De Janeiro, 20030-130')

(39.78373, -100.445882)